In [ ]:
import argparse

In [ ]:
def load_data(file_path):
    # TODO: Load data from CSV file

    return df

In [ ]:
def clean_data(df):
    # TODO: Handle missing values, outliers, etc.

    return df_clean

In [ ]:
def preprocess_data(df):
    # TODO: Generate new features, transform existing features, resampling, etc.

    return df_processed

In [ ]:
def save_data(df, output_file):
    # TODO: Save processed data to a CSV file
    pass

In [ ]:
def parse_arguments():
    parser = argparse.ArgumentParser(description='Data processing script for Energy Forecasting Hackathon')
    parser.add_argument(
        '--input_file',
        type=str,
        default='data/raw_data.csv',
        help='Path to the raw data file to process'
    )
    parser.add_argument(
        '--output_file', 
        type=str, 
        default='data/processed_data.csv', 
        help='Path to save the processed data'
    )
    return parser.parse_args()

In [ ]:
def main(input_file, output_file):
    df = load_data(input_file)
    df_clean = clean_data(df)
    df_processed = preprocess_data(df_clean)
    save_data(df_processed, output_file)

In [ ]:
if __name__ == "__main__":
    args = parse_arguments()
    main(args.input_file, args.output_file)

* Data processing for energy generation data
* data resampled to an hourly level

In [1]:
import pandas as pd
import numpy as np
import os

In [2]:
file_name = "uk/gen_UK_B19.csv"

In [3]:
df_gen = pd.read_csv(f"/kaggle/input/europe-green-electricity-generation-consumption/2022-2023/{file_name}")
df_gen

,StartTime,EndTime,AreaID,UnitName,PsrType,quantity
0,2022-01-26T23:30+00:00Z,2022-01-27T00:00+00:00Z,10Y1001A1001A92E,MAW,B19,433
1,2022-01-27T00:00+00:00Z,2022-01-27T00:30+00:00Z,10Y1001A1001A92E,MAW,B19,472
2,2022-01-27T00:30+00:00Z,2022-01-27T01:00+00:00Z,10Y1001A1001A92E,MAW,B19,549
3,2022-01-27T01:00+00:00Z,2022-01-27T01:30+00:00Z,10Y1001A1001A92E,MAW,B19,520
4,2022-01-27T01:30+00:00Z,2022-01-27T02:00+00:00Z,10Y1001A1001A92E,MAW,B19,407
...,...,...,...,...,...,...
12639,2022-12-31T21:00+00:00Z,2022-12-31T21:30+00:00Z,NaN,MAW,B19,183
12640,2022-12-31T21:30+00:00Z,2022-12-31T22:00+00:00Z,NaN,MAW,B19,155
12641,2022-12-31T22:00+00:00Z,2022-12-31T22:30+00:00Z,NaN,MAW,B19,132
12642,2022-12-31T22:30+00:00Z,2022-12-31T23:00+00:00Z,NaN,MAW,B19,112


In [4]:
df_gen.dtypes

StartTime    object
EndTime      object
AreaID       object
UnitName     object
PsrType      object
quantity      int64
dtype: object

In [ ]:
file_name = "gen_UK_B19_data_profile.html"

In [ ]:
from ydata_profiling import ProfileReport
df_gen_data_profile_path = os.path.join('/kaggle/working/', file_name)
profile = ProfileReport(df_gen, title="Energy Generation UK B19")
profile.to_file(df_gen_data_profile_path)

In [5]:
df_gen = df_gen.drop(labels=["AreaID", "UnitName", "PsrType"], axis=1)
df_gen

,StartTime,EndTime,quantity
0,2022-01-26T23:30+00:00Z,2022-01-27T00:00+00:00Z,433
1,2022-01-27T00:00+00:00Z,2022-01-27T00:30+00:00Z,472
2,2022-01-27T00:30+00:00Z,2022-01-27T01:00+00:00Z,549
3,2022-01-27T01:00+00:00Z,2022-01-27T01:30+00:00Z,520
4,2022-01-27T01:30+00:00Z,2022-01-27T02:00+00:00Z,407
...,...,...,...
12639,2022-12-31T21:00+00:00Z,2022-12-31T21:30+00:00Z,183
12640,2022-12-31T21:30+00:00Z,2022-12-31T22:00+00:00Z,155
12641,2022-12-31T22:00+00:00Z,2022-12-31T22:30+00:00Z,132
12642,2022-12-31T22:30+00:00Z,2022-12-31T23:00+00:00Z,112


In [6]:
from datetime import datetime, timedelta, timezone

df_gen_resampling = pd.DataFrame(data=[], columns=["StartTime", "EndTime", "quantity"])

#num_time_slot_in_hour = 4
num_time_slot_in_hour = 2
#interval_min_in_hour = 15
interval_min_in_hour = 30
interval_count = 0
quantity = 0
index = 0
indx = 0
start_date_1 = datetime.now()
start_date = datetime.now()
pre_end_date = datetime.now()
next_start_date = datetime.now()
interval_min = timedelta(minutes=interval_min_in_hour)
interval_hour = timedelta(hours=1)

while index < df_gen.shape[0]:
    row = df_gen.loc[index]
    index = index + 1
    if interval_count == num_time_slot_in_hour: 
        start_date_1 = datetime(start_date_1.year, start_date_1.month, start_date_1.day, start_date_1.hour, tzinfo=start_date_1.tzinfo)
        df_gen_resampling.loc[indx, "StartTime"] = start_date_1
        df_gen_resampling.loc[indx, "EndTime"] = start_date_1 + interval_hour
        df_gen_resampling.loc[indx, "quantity"] = quantity
        interval_count = 0
        quantity = 0
        indx = indx + 1
    
    start_date = datetime.fromisoformat(str(row["StartTime"]).replace("Z", ""))
    row_end_date = datetime.fromisoformat(str(row["EndTime"]).replace("Z", ""))
    end_date = start_date + interval_min
    
    if interval_count == 0:
        start_date_1 = start_date
        interval_count = interval_count + 1
        if end_date == row_end_date:
            quantity = quantity + row["quantity"]
    else:
        iter_count = 0
        while interval_count < num_time_slot_in_hour:
            if iter_count == 0:
                pre_end_date = datetime.fromisoformat(str(df_gen.loc[index - 2, "EndTime"]).replace("Z", ""))
            else:
                pre_end_date = pre_end_date + interval_min
            iter_count = iter_count + 1
            pre_start_date = pre_end_date - interval_min
            interval_count = interval_count + 1
            if start_date == pre_end_date and start_date.hour == pre_start_date.hour:
                if end_date > pre_end_date and end_date == row_end_date:
                    quantity = quantity + row["quantity"]
                    break
            if interval_count == num_time_slot_in_hour:
                index = index - 1

In [7]:
start_date_1 = datetime(start_date_1.year, start_date_1.month, start_date_1.day, start_date_1.hour, tzinfo=start_date_1.tzinfo)
df_gen_resampling.loc[indx, "StartTime"] = start_date_1
df_gen_resampling.loc[indx, "EndTime"] = start_date_1 + interval_hour
df_gen_resampling.loc[indx, "quantity"] = quantity

In [8]:
df_gen_resampling

,StartTime,EndTime,quantity
0,2022-01-26 23:00:00+00:00,2022-01-27 00:00:00+00:00,433
1,2022-01-27 00:00:00+00:00,2022-01-27 01:00:00+00:00,1021
2,2022-01-27 01:00:00+00:00,2022-01-27 02:00:00+00:00,927
3,2022-01-27 02:00:00+00:00,2022-01-27 03:00:00+00:00,673
4,2022-01-27 03:00:00+00:00,2022-01-27 04:00:00+00:00,426
...,...,...,...
6463,2022-12-31 19:00:00+00:00,2022-12-31 20:00:00+00:00,412
6464,2022-12-31 20:00:00+00:00,2022-12-31 21:00:00+00:00,415
6465,2022-12-31 21:00:00+00:00,2022-12-31 22:00:00+00:00,338
6466,2022-12-31 22:00:00+00:00,2022-12-31 23:00:00+00:00,244


In [9]:
file_name = "gen_UK_B19_resampling.csv"
df_gen_resampling.to_csv(f"/kaggle/working/{file_name}", index=False)

* data resampled to monthly level

In [10]:
import pandas as pd
import numpy as np
import os

In [11]:
def is_file_exist(filedir, filename):
    file_exist = False
    #for _, _, filenames in os.walk('/kaggle/working'):
    for _, _, filenames in os.walk(filedir):
        if filename in filenames:
            file_exist = True
            break
    return file_exist

In [12]:
file_name = "uk/gen_UK_B19.csv"

In [13]:
df_gen = pd.read_csv(f"/kaggle/input/europe-green-electricity-generation-consumption/2022-2023/{file_name}")
df_gen

,StartTime,EndTime,AreaID,UnitName,PsrType,quantity
0,2022-01-26T23:30+00:00Z,2022-01-27T00:00+00:00Z,10Y1001A1001A92E,MAW,B19,433
1,2022-01-27T00:00+00:00Z,2022-01-27T00:30+00:00Z,10Y1001A1001A92E,MAW,B19,472
2,2022-01-27T00:30+00:00Z,2022-01-27T01:00+00:00Z,10Y1001A1001A92E,MAW,B19,549
3,2022-01-27T01:00+00:00Z,2022-01-27T01:30+00:00Z,10Y1001A1001A92E,MAW,B19,520
4,2022-01-27T01:30+00:00Z,2022-01-27T02:00+00:00Z,10Y1001A1001A92E,MAW,B19,407
...,...,...,...,...,...,...
12639,2022-12-31T21:00+00:00Z,2022-12-31T21:30+00:00Z,NaN,MAW,B19,183
12640,2022-12-31T21:30+00:00Z,2022-12-31T22:00+00:00Z,NaN,MAW,B19,155
12641,2022-12-31T22:00+00:00Z,2022-12-31T22:30+00:00Z,NaN,MAW,B19,132
12642,2022-12-31T22:30+00:00Z,2022-12-31T23:00+00:00Z,NaN,MAW,B19,112


In [14]:
from datetime import datetime, timedelta, timezone
start_date = datetime.fromisoformat(str(df_gen.loc[0, "StartTime"]).replace("Z", ""))
if start_date.year == 2021:
    df_gen = df_gen.drop(0)
    df_gen = df_gen.set_index(pd.Index(np.arange(0, df_gen.shape[0])))
df_gen

,StartTime,EndTime,AreaID,UnitName,PsrType,quantity
0,2022-01-26T23:30+00:00Z,2022-01-27T00:00+00:00Z,10Y1001A1001A92E,MAW,B19,433
1,2022-01-27T00:00+00:00Z,2022-01-27T00:30+00:00Z,10Y1001A1001A92E,MAW,B19,472
2,2022-01-27T00:30+00:00Z,2022-01-27T01:00+00:00Z,10Y1001A1001A92E,MAW,B19,549
3,2022-01-27T01:00+00:00Z,2022-01-27T01:30+00:00Z,10Y1001A1001A92E,MAW,B19,520
4,2022-01-27T01:30+00:00Z,2022-01-27T02:00+00:00Z,10Y1001A1001A92E,MAW,B19,407
...,...,...,...,...,...,...
12639,2022-12-31T21:00+00:00Z,2022-12-31T21:30+00:00Z,NaN,MAW,B19,183
12640,2022-12-31T21:30+00:00Z,2022-12-31T22:00+00:00Z,NaN,MAW,B19,155
12641,2022-12-31T22:00+00:00Z,2022-12-31T22:30+00:00Z,NaN,MAW,B19,132
12642,2022-12-31T22:30+00:00Z,2022-12-31T23:00+00:00Z,NaN,MAW,B19,112


In [15]:
from datetime import datetime, timedelta, timezone
import calendar
country = "UK"
country_id = 1
#df_gen = df_gen.loc[1:]
file_exist = is_file_exist("/kaggle/input/europe-green-electricity-generation-consumption", "gen_2022_monthly.csv")
if file_exist == True:
    df_gen_2022_monthly = pd.read_csv("/kaggle/input/europe-green-electricity-generation-consumption/gen_2022_monthly.csv")
    indx = df_gen_2022_monthly.shape[0]
else:
    df_gen_2022_monthly = pd.DataFrame(data=[], columns=["StartTime", "EndTime", "PsrType", "PsrTypeName", "GenType", "AreaID", "Country", "CountryID", "quantity", "UnitName"])
    indx = 0
index = 0
init_date = datetime.fromisoformat(str(df_gen.loc[0, "StartTime"]).replace("Z", ""))
month = init_date.month
start_hour = init_date.hour
start_min = init_date.minute
quantity = 0
start_date = datetime.now()
pre_start_date = datetime.now()
area_id = ""
while index < df_gen.shape[0]:
    row = df_gen.loc[index]
    index = index + 1
    start_date = datetime.fromisoformat(str(row["StartTime"]).replace("Z", ""))
    if start_date.hour == start_hour and start_date.minute == start_min:
        area_id = row["AreaID"]
    if start_date.month == month:
        quantity = quantity + row["quantity"]
    else:
        pre_start_date = datetime.fromisoformat(str(df_gen.loc[index - 2, "StartTime"]).replace("Z", ""))
        pre_start_date = datetime(pre_start_date.year, pre_start_date.month, 1, tzinfo=start_date.tzinfo)
        df_gen_2022_monthly.loc[indx, "StartTime"] = pre_start_date
        days_in_month = calendar.monthrange(pre_start_date.year, pre_start_date.month)[1]
        interval_month = timedelta(days=days_in_month - 1)
        df_gen_2022_monthly.loc[indx, "EndTime"] = pre_start_date + interval_month
        df_gen_2022_monthly.loc[indx, "PsrType"] = row["PsrType"]
        df_gen_2022_monthly.loc[indx, "AreaID"] = area_id
        df_gen_2022_monthly.loc[indx, "Country"] = country
        df_gen_2022_monthly.loc[indx, "CountryID"] = country_id
        df_gen_2022_monthly.loc[indx, "quantity"] = quantity
        df_gen_2022_monthly.loc[indx, "UnitName"] = row["UnitName"]
        
        if row["PsrType"] == "B18" or row["PsrType"] == "B19":
            df_gen_2022_monthly.loc[indx, "GenType"] = "Wind"
        elif row["PsrType"] == "B16":
            df_gen_2022_monthly.loc[indx, "GenType"] = "Solar"
        elif row["PsrType"] == "B10" or row["PsrType"] == "B11" or row["PsrType"] == "B12":
            df_gen_2022_monthly.loc[indx, "GenType"] = "Water"
        elif row["PsrType"] == "B01" or row["PsrType"] == "B09" or row["PsrType"] == "B15":
            df_gen_2022_monthly.loc[indx, "GenType"] = "Other"

        if row["PsrType"] == "B18":
            df_gen_2022_monthly.loc[indx, "PsrTypeName"] = "Wind Offshore"
        elif row["PsrType"] == "B19":
            df_gen_2022_monthly.loc[indx, "PsrTypeName"] = "Wind Onshore"
        elif row["PsrType"] == "B16":
            df_gen_2022_monthly.loc[indx, "PsrTypeName"] = "Solar"
        elif row["PsrType"] == "B10":
            df_gen_2022_monthly.loc[indx, "PsrTypeName"] = "Hydro Pumped Storage"
        elif row["PsrType"] == "B11":
            df_gen_2022_monthly.loc[indx, "PsrTypeName"] = "Hydro Run-of-river and poundage"
        elif row["PsrType"] == "B12":
            df_gen_2022_monthly.loc[indx, "PsrTypeName"] = "Hydro Water Reservoir"
        elif row["PsrType"] == "B01":
            df_gen_2022_monthly.loc[indx, "PsrTypeName"] = "Biomass"
        elif row["PsrType"] == "B09":
            df_gen_2022_monthly.loc[indx, "PsrTypeName"] = "Geothermal"
        elif row["PsrType"] == "B15":
            df_gen_2022_monthly.loc[indx, "PsrTypeName"] = "Other renewable"
        
        indx = indx + 1
        month = start_date.month
        next_row = datetime.fromisoformat(str(df_gen.loc[index, "StartTime"]).replace("Z", ""))
        start_hour = next_row.hour
        start_min = next_row.minute
        quantity = row["quantity"]

In [16]:
pre_start_date = datetime.fromisoformat(str(df_gen.loc[index - 2, "StartTime"]).replace("Z", ""))
pre_start_date = datetime(pre_start_date.year, pre_start_date.month, 1, tzinfo=start_date.tzinfo)
df_gen_2022_monthly.loc[indx, "StartTime"] = pre_start_date
days_in_month = calendar.monthrange(pre_start_date.year, pre_start_date.month)[1]
interval_month = timedelta(days=days_in_month - 1)
df_gen_2022_monthly.loc[indx, "EndTime"] = pre_start_date + interval_month
df_gen_2022_monthly.loc[indx, "PsrType"] = row["PsrType"]
df_gen_2022_monthly.loc[indx, "AreaID"] = area_id
df_gen_2022_monthly.loc[indx, "Country"] = country
df_gen_2022_monthly.loc[indx, "CountryID"] = country_id
df_gen_2022_monthly.loc[indx, "quantity"] = quantity
df_gen_2022_monthly.loc[indx, "UnitName"] = row["UnitName"]

if row["PsrType"] == "B18" or row["PsrType"] == "B19":
    df_gen_2022_monthly.loc[indx, "GenType"] = "Wind"
elif row["PsrType"] == "B16":
    df_gen_2022_monthly.loc[indx, "GenType"] = "Solar"
elif row["PsrType"] == "B10" or row["PsrType"] == "B11" or row["PsrType"] == "B12" or row["PsrType"] == "B13":
    df_gen_2022_monthly.loc[indx, "GenType"] = "Water"
elif row["PsrType"] == "B01" or row["PsrType"] == "B09" or row["PsrType"] == "B15":
    df_gen_2022_monthly.loc[indx, "GenType"] = "Other"

if row["PsrType"] == "B18":
    df_gen_2022_monthly.loc[indx, "PsrTypeName"] = "Wind Offshore"
elif row["PsrType"] == "B19":
    df_gen_2022_monthly.loc[indx, "PsrTypeName"] = "Wind Onshore"
elif row["PsrType"] == "B16":
    df_gen_2022_monthly.loc[indx, "PsrTypeName"] = "Solar"
elif row["PsrType"] == "B10":
    df_gen_2022_monthly.loc[indx, "PsrTypeName"] = "Hydro Pumped Storage"
elif row["PsrType"] == "B11":
    df_gen_2022_monthly.loc[indx, "PsrTypeName"] = "Hydro Run-of-river and poundage"
elif row["PsrType"] == "B12":
    df_gen_2022_monthly.loc[indx, "PsrTypeName"] = "Hydro Water Reservoir"
elif row["PsrType"] == "B01":
    df_gen_2022_monthly.loc[indx, "PsrTypeName"] = "Biomass"
elif row["PsrType"] == "B09":
    df_gen_2022_monthly.loc[indx, "PsrTypeName"] = "Geothermal"
elif row["PsrType"] == "B13":
    df_gen_2022_monthly.loc[indx, "PsrTypeName"] = "Marine"    
elif row["PsrType"] == "B15":
    df_gen_2022_monthly.loc[indx, "PsrTypeName"] = "Other renewable"

In [17]:
df_gen_2022_monthly

,StartTime,EndTime,PsrType,AreaID,Country,CountryID,quantity,UnitName,GenType,PsrTypeName
0,2022-01-01 00:00:00+00:00,2022-01-31 00:00:00+00:00,B01,10Y1001A1001A83F,DE,2.0,13777505.0,nan,nan,nan
1,2022-02-01 00:00:00+00:00,2022-02-28 00:00:00+00:00,B01,10Y1001A1001A83F,DE,2.0,12540609.0,nan,nan,nan
2,2022-03-01 00:00:00+00:00,2022-03-31 00:00:00+00:00,B01,10Y1001A1001A83F,DE,2.0,13949928.0,nan,nan,nan
3,2022-04-01 00:00:00+00:00,2022-04-30 00:00:00+00:00,B01,10Y1001A1001A83F,DE,2.0,13169383.0,nan,nan,nan
4,2022-05-01 00:00:00+00:00,2022-05-31 00:00:00+00:00,B01,10Y1001A1001A83F,DE,2.0,13334915.0,nan,nan,nan
...,...,...,...,...,...,...,...,...,...,...
753,2022-03-01 00:00:00+00:00,2022-03-31 00:00:00+00:00,B19,NaN,UK,1.0,479952.0,MAW,Wind,Wind Onshore
754,2022-04-01 00:00:00+00:00,2022-04-30 00:00:00+00:00,B19,NaN,UK,1.0,27040.0,MAW,Wind,Wind Onshore
755,2022-10-01 00:00:00+00:00,2022-10-31 00:00:00+00:00,B19,NaN,UK,1.0,177457.0,MAW,Wind,Wind Onshore
756,2022-11-01 00:00:00+00:00,2022-11-30 00:00:00+00:00,B19,NaN,UK,1.0,577406.0,MAW,Wind,Wind Onshore


In [18]:
df_gen_2022_monthly.to_csv(f"/kaggle/working/gen_2022_monthly.csv", index=False)